# Lab 1: EMG Signal Processing and Analysis

## Learning Objectives

By the end of this lab, you will be able to:

1. **Understand LSL architecture** - how data flows from devices through the network
2. **Load and explore EMG data** - work with recorded physiological signals
3. **Apply signal processing techniques** - filtering, rectification, envelope extraction
4. **Extract meaningful features** - time and frequency domain analysis
5. **Visualize and interpret results** - create publication-quality plots

---

## Setup and Imports

First, let's import the necessary libraries and configure our environment.

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
from pathlib import Path

# Signal processing
import scipy.signal as signal
from scipy import stats

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline

# For interactive plots (optional - uncomment if using)
# %matplotlib widget

# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Our custom EMG processing module
import sys
sys.path.insert(0, '../src')
from emg_processing import (
    bandpass_filter, notch_filter, rectify, envelope, rms,
    compute_features, power_spectral_density, process_emg_pipeline
)

print("✓ All imports successful!")

## Part 1: Understanding the Data

### 1.1 Load the Sample Data

We'll start by loading sample EMG data collected from the Myo Armband. This data contains 8 channels of surface EMG (sEMG) sampled at 200 Hz.

In [ ]:
# Load the sample data
# Replace this path with your own data file if needed
data_path = Path('../data/sample_emg.csv')

# Check if sample data exists, if not create synthetic data for demo
if data_path.exists():
    df = pd.read_csv(data_path)
    print(f"Loaded data from {data_path}")
else:
    print("Sample data not found. Creating synthetic EMG data for demonstration...")
    
    # Create synthetic EMG data
    np.random.seed(42)
    sample_rate = 200  # Hz (Myo sample rate)
    duration = 30  # seconds
    n_samples = sample_rate * duration
    
    # Time array
    timestamps = np.arange(n_samples) / sample_rate
    
    # Create 8 channels of EMG data
    data = {'timestamp': timestamps}
    
    for ch in range(1, 9):
        # Baseline noise
        noise = np.random.randn(n_samples) * 5
        
        # Add some muscle activation "bursts"
        activation = np.zeros(n_samples)
        
        # Simulate gestures at different times
        for gesture_time in [5, 10, 15, 20, 25]:
            start = int(gesture_time * sample_rate)
            end = int((gesture_time + 2) * sample_rate)
            if end < n_samples:
                # Different channels activate differently for each gesture
                amplitude = 30 + 20 * np.sin(ch + gesture_time)
                activation[start:end] = amplitude * (1 + 0.3 * np.random.randn(end-start))
        
        # Combine and add 60 Hz power line noise
        t = timestamps
        power_line = 3 * np.sin(2 * np.pi * 60 * t)
        
        data[f'EMG_{ch}'] = noise + activation + power_line
    
    df = pd.DataFrame(data)
    
    # Save for future use
    data_path.parent.mkdir(exist_ok=True)
    df.to_csv(data_path, index=False)
    print(f"Created and saved synthetic data to {data_path}")

# Display basic info
print(f"\nDataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head()

### 1.2 Explore the Data

Let's get some basic statistics and understand what we're working with.

In [ ]:
# Get descriptive statistics for EMG channels
emg_columns = [col for col in df.columns if 'EMG' in col]
print("EMG Channel Statistics:")
print("=" * 50)
df[emg_columns].describe()

In [ ]:
# Calculate recording duration and sample rate
if 'timestamp' in df.columns:
    duration = df['timestamp'].iloc[-1] - df['timestamp'].iloc[0]
    sample_rate = len(df) / duration
    print(f"Recording duration: {duration:.2f} seconds")
    print(f"Estimated sample rate: {sample_rate:.1f} Hz")
    print(f"Total samples: {len(df)}")
else:
    # Assume Myo sample rate
    sample_rate = 200
    duration = len(df) / sample_rate
    print(f"Assumed sample rate: {sample_rate} Hz")
    print(f"Estimated duration: {duration:.2f} seconds")

### 1.3 Visualize Raw EMG Signals

Let's plot all 8 EMG channels to see what the raw signals look like.

In [ ]:
# Create time axis
if 'timestamp' in df.columns:
    time = df['timestamp'].values
else:
    time = np.arange(len(df)) / sample_rate

# Plot all EMG channels
fig, axes = plt.subplots(4, 2, figsize=(14, 10), sharex=True)
axes = axes.flatten()

for i, col in enumerate(emg_columns):
    axes[i].plot(time, df[col], linewidth=0.5, alpha=0.8)
    axes[i].set_ylabel(f'{col}')
    axes[i].set_title(f'Channel {i+1}')

axes[-2].set_xlabel('Time (s)')
axes[-1].set_xlabel('Time (s)')

plt.suptitle('Raw EMG Signals - All Channels', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

**Question 1:** Looking at the raw EMG plots above, can you identify:
- Which time periods show muscle activation (higher amplitude)?
- Do all channels show similar activation patterns?
- Can you spot any high-frequency noise or interference?

*Write your observations here:*

> YOUR ANSWER HERE

---

## Part 2: Signal Processing

Raw EMG signals contain noise and artifacts that we need to remove. We'll apply a processing pipeline:

1. **Bandpass filter**: Keep frequencies relevant to EMG (20-95 Hz for Myo)
2. **Notch filter**: Remove 60 Hz power line interference
3. **Rectification**: Take absolute value
4. **Envelope extraction**: Smooth to get activation level

### 2.1 Select a Channel for Processing

In [ ]:
# Select channel 1 for detailed analysis
channel = 'EMG_1'
raw_signal = df[channel].values.astype(float)

# Remove DC offset (center around zero)
raw_signal = raw_signal - np.mean(raw_signal)

print(f"Processing {channel}")
print(f"Signal length: {len(raw_signal)} samples")
print(f"Duration: {len(raw_signal)/sample_rate:.2f} seconds")

### 2.2 Apply Bandpass Filter

EMG signals typically have useful content between 20-500 Hz. For the Myo (200 Hz sample rate), we'll use 20-95 Hz to stay below the Nyquist frequency.

In [ ]:
# Apply bandpass filter
low_cutoff = 20  # Hz
high_cutoff = 95  # Hz (must be < sample_rate/2 = 100 Hz)

filtered_signal = bandpass_filter(
    raw_signal, 
    low_freq=low_cutoff, 
    high_freq=high_cutoff,
    sample_rate=sample_rate
)

print(f"Applied bandpass filter: {low_cutoff}-{high_cutoff} Hz")

### 2.3 Apply Notch Filter

Power line interference (60 Hz in US, 50 Hz in Europe) is a common source of noise.

In [ ]:
# Apply notch filter at 60 Hz
notch_freq = 60  # Hz

filtered_notched = notch_filter(
    filtered_signal,
    notch_freq=notch_freq,
    quality_factor=30,
    sample_rate=sample_rate
)

print(f"Applied notch filter at {notch_freq} Hz")

### 2.4 Compare Raw vs Filtered Signals

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)

# Select a shorter window for better visualization
window_start = 5  # seconds
window_end = 10  # seconds
start_idx = int(window_start * sample_rate)
end_idx = int(window_end * sample_rate)

t_window = time[start_idx:end_idx]

axes[0].plot(t_window, raw_signal[start_idx:end_idx], 'b', linewidth=0.5, alpha=0.8)
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Raw EMG Signal')
axes[0].legend(['Raw'], loc='upper right')

axes[1].plot(t_window, filtered_notched[start_idx:end_idx], 'g', linewidth=0.5, alpha=0.8)
axes[1].set_ylabel('Amplitude')
axes[1].set_xlabel('Time (s)')
axes[1].set_title('Filtered EMG Signal (Bandpass + Notch)')
axes[1].legend(['Filtered'], loc='upper right')

plt.suptitle(f'{channel}: Raw vs Filtered', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 2.5 Rectification and Envelope Extraction

To get the activation level, we rectify (take absolute value) and then smooth the signal.

In [ ]:
# Rectify the signal
rectified_signal = rectify(filtered_notched)

# Extract envelope
envelope_signal = envelope(filtered_notched, cutoff_freq=6, sample_rate=sample_rate)

# Also compute RMS
window_size = int(0.1 * sample_rate)  # 100ms window
rms_signal = rms(filtered_notched, window_size=window_size)

print("Computed rectified signal, envelope, and RMS")

In [ ]:
# Plot the complete processing pipeline
fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)

axes[0].plot(time, raw_signal, 'b', linewidth=0.5, alpha=0.7)
axes[0].set_ylabel('Raw')
axes[0].set_title('Step 1: Raw EMG')

axes[1].plot(time, filtered_notched, 'g', linewidth=0.5, alpha=0.7)
axes[1].set_ylabel('Filtered')
axes[1].set_title('Step 2: Bandpass + Notch Filtered')

axes[2].plot(time, rectified_signal, 'orange', linewidth=0.5, alpha=0.7)
axes[2].set_ylabel('Rectified')
axes[2].set_title('Step 3: Rectified (Absolute Value)')

axes[3].plot(time, envelope_signal, 'r', linewidth=1.5, label='Envelope')
axes[3].plot(time, rms_signal, 'm', linewidth=1.5, alpha=0.7, label='RMS')
axes[3].set_ylabel('Amplitude')
axes[3].set_xlabel('Time (s)')
axes[3].set_title('Step 4: Envelope and RMS')
axes[3].legend()

plt.suptitle(f'{channel}: EMG Processing Pipeline', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

**Question 2:** Compare the envelope and RMS signals in the bottom plot.
- What do these signals represent?
- When would you use envelope vs RMS?
- Can you identify the muscle activation periods from the envelope?

*Write your answer here:*

> YOUR ANSWER HERE

---

## Part 3: Frequency Domain Analysis

The Power Spectral Density (PSD) shows how signal power is distributed across frequencies.

In [ ]:
# Compute PSD for raw and filtered signals
freqs_raw, psd_raw = power_spectral_density(raw_signal, sample_rate)
freqs_filt, psd_filt = power_spectral_density(filtered_notched, sample_rate)

# Plot
fig, ax = plt.subplots(figsize=(12, 5))

ax.semilogy(freqs_raw, psd_raw, 'b', alpha=0.7, label='Raw')
ax.semilogy(freqs_filt, psd_filt, 'g', alpha=0.9, label='Filtered')

# Mark important frequencies
ax.axvline(x=60, color='r', linestyle='--', alpha=0.5, label='60 Hz (power line)')
ax.axvspan(20, 95, alpha=0.1, color='green', label='Passband (20-95 Hz)')

ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power Spectral Density (V²/Hz)')
ax.set_title(f'{channel}: Power Spectral Density')
ax.legend()
ax.set_xlim([0, 100])

plt.tight_layout()
plt.show()

**Question 3:** Looking at the PSD plot:
- What happened to the 60 Hz component after filtering?
- Why is the log scale (semilogy) useful for viewing PSD?
- What frequency range contains most of the EMG signal power?

*Write your answer here:*

> YOUR ANSWER HERE

---

## Part 4: Feature Extraction

For gesture classification or control applications, we extract features from the EMG signals.

In [ ]:
# Compute features for the entire signal
features = compute_features(filtered_notched, sample_rate)

print(f"Features for {channel}:")
print("=" * 40)
for name, value in features.items():
    print(f"{name:15s}: {value:12.4f}")

### 4.1 Compare Features Across Channels

In [ ]:
# Compute features for all channels
all_features = []

for col in emg_columns:
    signal_data = df[col].values.astype(float)
    signal_data = signal_data - np.mean(signal_data)
    
    # Filter
    filtered = bandpass_filter(signal_data, 20, 95, sample_rate)
    filtered = notch_filter(filtered, 60, sample_rate=sample_rate)
    
    # Extract features
    features = compute_features(filtered, sample_rate)
    features['channel'] = col
    all_features.append(features)

features_df = pd.DataFrame(all_features)
features_df = features_df.set_index('channel')

print("Features across all channels:")
features_df.round(4)

In [ ]:
# Visualize feature comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# RMS by channel
axes[0, 0].bar(range(1, 9), features_df['rms'])
axes[0, 0].set_xlabel('Channel')
axes[0, 0].set_ylabel('RMS')
axes[0, 0].set_title('RMS by Channel')

# Mean Absolute Value by channel
axes[0, 1].bar(range(1, 9), features_df['mav'], color='orange')
axes[0, 1].set_xlabel('Channel')
axes[0, 1].set_ylabel('MAV')
axes[0, 1].set_title('Mean Absolute Value by Channel')

# Mean frequency by channel
axes[1, 0].bar(range(1, 9), features_df['mean_freq'], color='green')
axes[1, 0].set_xlabel('Channel')
axes[1, 0].set_ylabel('Frequency (Hz)')
axes[1, 0].set_title('Mean Frequency by Channel')

# Total power by channel
axes[1, 1].bar(range(1, 9), features_df['total_power'], color='purple')
axes[1, 1].set_xlabel('Channel')
axes[1, 1].set_ylabel('Power')
axes[1, 1].set_title('Total Power by Channel')

plt.tight_layout()
plt.show()

---

## Part 5: Your Turn!

Now it's your turn to apply what you've learned. Complete the following exercises:

### Exercise 1: Process Your Own Data

Load your recorded EMG data and apply the processing pipeline.

In [ ]:
# TODO: Load your data file
# my_data = pd.read_csv('path/to/your/data.csv')

# YOUR CODE HERE


### Exercise 2: Segment Your Data by Gesture

If you recorded Rock, Paper, Scissors gestures, segment the data for each gesture type.

In [ ]:
# TODO: Define your gesture timestamps
# Example:
# gesture_times = {
#     'rock': [(5.0, 7.0), (15.0, 17.0), ...],  # (start, end) in seconds
#     'paper': [(8.0, 10.0), ...],
#     'scissors': [(11.0, 13.0), ...]
# }

# YOUR CODE HERE


### Exercise 3: Compare Gestures

Extract features for each gesture type and compare them.

In [ ]:
# TODO: Extract features for each gesture
# Compare the features across gestures
# Which features best distinguish the gestures?

# YOUR CODE HERE


---

## Summary

In this lab, you learned:

1. **EMG signals** contain information about muscle activation
2. **Filtering** removes noise while preserving useful signal content
3. **Envelope extraction** gives you the activation level over time
4. **Feature extraction** converts continuous signals into useful numbers
5. **Frequency analysis** reveals the spectral content of signals

These techniques are fundamental to many applications including:
- Myoelectric prosthesis control
- Gesture recognition
- Muscle fatigue analysis
- Rehabilitation monitoring

---

## References

- Merletti, R., & Parker, P. J. (2004). Electromyography: Physiology, Engineering, and Non-Invasive Applications
- De Luca, C. J. (1997). The use of surface electromyography in biomechanics
- Lab Streaming Layer: https://labstreaminglayer.org/